In [6]:
import pandas as pd
import pyodbc
import sqlite3

# iedereen heeft andere servername btw

In [16]:
servername = 'DESKTOP-G20KDR6\SQLEXPRESS'

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\faaru\AppData\Local\Temp\ipykernel_16112\237110110.py:1: SyntaxWarning: invalid escape sequence '\S'
  servername = 'DESKTOP-G20KDR6\SQLEXPRESS'


# Northwind connectie

In [17]:
northwind_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=NorthWind; Trusted_Connection=yes')
nwCursor = northwind_conn.cursor()
nwCursor

# Adventureworks connectie

In [18]:
adventureworks_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=AdventureWorks2019; Trusted_Connection=yes')
awCursor = adventureworks_conn.cursor()
awCursor

# Aenc connectie

In [19]:
aenc_conn = pyodbc.connect('DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../raw-databases/aenc.accdb;')
aencCursor = aenc_conn.cursor()

table_names = [table_info.table_name for table_info in aencCursor.tables()]

# Print table names
print(table_names)

['MSysAccessStorage', 'MSysACEs', 'MSysComplexColumns', 'MSysIMEXColumns', 'MSysIMEXSpecs', 'MSysNameMap', 'MSysNavPaneGroupCategories', 'MSysNavPaneGroups', 'MSysNavPaneGroupToObjects', 'MSysNavPaneObjectIDs', 'MSysObjects', 'MSysQueries', 'MSysRelationships', 'MSysResources', 'bonus', 'customer', 'department', 'employee', 'product', 'region', 'sales_order', 'sales_order_item', 'state']


In [21]:
aenctest = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
awtest = pd.read_sql_query('SELECT * FROM Person.Address', adventureworks_conn)
nwtest = pd.read_sql_query('SELECT * FROM customers', northwind_conn)

# print(aenctest)
# print(awtest)
# print(nwtest)

C:\Users\faaru\AppData\Local\Temp\ipykernel_16112\3893602639.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  aenctest = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
C:\Users\faaru\AppData\Local\Temp\ipykernel_16112\3893602639.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  awtest = pd.read_sql_query('SELECT * FROM Person.Address', adventureworks_conn)
C:\Users\faaru\AppData\Local\Temp\ipykernel_16112\3893602639.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nwtest = pd.read_sql_query('SELECT * FROM cu

      id     fname         lname                 address         city state  \
0    101  Michaels        Devlin     3114 Pioneer Avenue   Rutherford    NJ   
1    102      Beth        Reiser      1033 Whippany Road     New York    NY   
2    103      Erin  Niedringhaus     1990 Windsor Street        Paoli    PA   
3    104    Meghan         Mason  550 Dundas Street East    Knoxville    TN   
4    105     Laura      McCarthy         1210 Highway 36       Carmel    IN   
..   ...       ...           ...                     ...          ...   ...   
121  552    Janice       O'Toole           654 West Hill    Nashville    TN   
122  553    Stevie      Nickolas    77 Recordings Circle       Tacoma    WA   
123  555   Philipe     Fernandez          99 Main Street  Los Angeles    CA   
124  661  Jennifer      Stutzman       3 Back Pages Lane      Missola    IL   
125  665   William      Thompson    91 Washington Street   Manchester    NY   

       zip       phone          company_name  
0   

AdventureWorks Person.Person / Person.Address / Person.PersonPhone

In [36]:
df_person = pd.read_sql_query('SELECT * FROM Person.Person', adventureworks_conn)
df_person_address = pd.read_sql_query('SELECT * FROM Person.Address', adventureworks_conn)
df_person_phone = pd.read_sql_query('SELECT * FROM Person.PersonPhone', adventureworks_conn)


C:\Users\faaru\AppData\Local\Temp\ipykernel_16112\653121990.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_person = pd.read_sql_query('SELECT * FROM Person.Person', adventureworks_conn)
C:\Users\faaru\AppData\Local\Temp\ipykernel_16112\653121990.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_person_address = pd.read_sql_query('SELECT * FROM Person.Address', adventureworks_conn)
C:\Users\faaru\AppData\Local\Temp\ipykernel_16112\653121990.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_person_phone = pd

aenc customer

In [53]:
df_aenc_customer = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
df_aenc_customer

C:\Users\faaru\AppData\Local\Temp\ipykernel_16112\3911038392.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_aenc_customer = pd.read_sql_query('SELECT * FROM customer', aenc_conn)


,id,fname,lname,address,city,state,zip,phone,company_name
0,101,Michaels,Devlin,3114 Pioneer Avenue,Rutherford,NJ,07070,2015558966,The Power Group
1,102,Beth,Reiser,1033 Whippany Road,New York,NY,10154,2125558725,AMF Corp.
2,103,Erin,Niedringhaus,1990 Windsor Street,Paoli,PA,19301,2155556513,Darling Associates
3,104,Meghan,Mason,550 Dundas Street East,Knoxville,TN,37919,6155555463,P.S.C.
4,105,Laura,McCarthy,1210 Highway 36,Carmel,IN,46032,3175558437,Amo & Sons
...,...,...,...,...,...,...,...,...,...
121,552,Janice,O'Toole,654 West Hill,Nashville,TN,37320,6155553689,Greensleeves
122,553,Stevie,Nickolas,77 Recordings Circle,Tacoma,WA,96521,5095551695,It's a Hit!
123,555,Philipe,Fernandez,99 Main Street,Los Angeles,CA,90205,2135554457,Quaker Fashions
124,661,Jennifer,Stutzman,3 Back Pages Lane,Missola,IL,60505,7085556857,Stutzman Advertising


NorthWind customers

In [38]:
df_northwind_customers = pd.read_sql_query('SELECT * FROM customers', northwind_conn)

C:\Users\faaru\AppData\Local\Temp\ipykernel_16112\2771955829.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_northwind_customers = pd.read_sql_query('SELECT * FROM customers', northwind_conn)


Specifieke Data eruit halen

In [67]:
# Select specific columns and rename them
df_person_phone_selected = df_person_phone[['BusinessEntityID', 'PhoneNumber']]
df_person_address_selected = df_person_address[['AddressLine1', 'City', 'PostalCode']]
df_northwind_customers_selected = df_northwind_customers[['CustomerID', 'CompanyName', 'Address', 'City', 'PostalCode', 'Phone']]
df_aenc_customer_selected = df_aenc_customer[['id', 'address', 'city', 'zip', 'phone', 'company_name']]

# show the first 5 rows of the dataframes
print(df_person_phone_selected.head())
print(df_person_address_selected.head())
print(df_northwind_customers_selected.head())
print(df_aenc_customer_selected.head())

   BusinessEntityID   PhoneNumber
0                 1  697-555-0142
1                 2  819-555-0175
2                 3  212-555-0187
3                 4  612-555-0100
4                 5  849-555-0139
           AddressLine1     City PostalCode
0         1970 Napa Ct.  Bothell      98011
1    9833 Mt. Dias Blv.  Bothell      98011
2  7484 Roundtree Drive  Bothell      98011
3      9539 Glenside Dr  Bothell      98011
4         1226 Shoe St.  Bothell      98011
  CustomerID                         CompanyName  \
0      ALFKI                 Alfreds Futterkiste   
1      ANATR  Ana Trujillo Emparedados y helados   
2      ANTON             Antonio Moreno Taquería   
3      AROUT                     Around the Horn   
4      BERGS                  Berglunds snabbköp   

                         Address         City PostalCode           Phone  
0                  Obere Str. 57       Berlin      12209     030-0074321  
1  Avda. de la Constitución 2222  México D.F.      05021    (5) 555-4

Samenvoegen in Customer DF

In [114]:
# Concatenate the dataframes along axis 0 to stack them vertically
customer = pd.concat([df_northwind_customers_selected, df_aenc_customer_selected], axis=0)
customer = pd.concat([customer, df_person_address_selected], axis=0)
customer = pd.concat([customer, df_person_phone_selected], axis=0)

# Assign unique identifiers to each row
customer['row_id'] = range(len(customer))

# Reassign values for columns as per the specified criteria
customer['id'] = customer.apply(lambda row: str(row['BusinessEntityID']) if not pd.isnull(row['BusinessEntityID']) else str(row['CustomerID']) if not pd.isnull(row['CustomerID']) else row['id'], axis=1)
customer['company_name'] = customer['company_name'].combine_first(customer['CompanyName'])
customer['address'] = customer['address'].combine_first(customer['Address']).combine_first(customer['AddressLine1'])
customer['city'] = customer['city'].combine_first(customer['City'])
customer['zip'] = customer['PostalCode'].combine_first(customer['zip'])
customer['phone'] = customer['phone'].combine_first(customer['PhoneNumber']).combine_first(customer['Phone'])

# Drop redundant columns
customer.drop(['BusinessEntityID', 'CustomerID', 'CompanyName', 'Address', 'AddressLine1', 'City', 'PostalCode', 'PhoneNumber', 'Phone'], axis=1, inplace=True)

# Reorder the columns
customer = customer[['id', 'company_name', 'address', 'city', 'zip', 'phone']]

# Display where id is 101
customer

,id,company_name,address,city,zip,phone
0,ALFKI,Alfreds Futterkiste,Obere Str. 57,Berlin,12209,030-0074321
1,ANATR,Ana Trujillo Emparedados y helados,Avda. de la Constitución 2222,México D.F.,05021,(5) 555-4729
2,ANTON,Antonio Moreno Taquería,Mataderos 2312,México D.F.,05023,(5) 555-3932
3,AROUT,Around the Horn,120 Hanover Sq.,London,WA1 1DP,(171) 555-7788
4,BERGS,Berglunds snabbköp,Berguvsvägen 8,Luleå,S-958 22,0921-12 34 65
...,...,...,...,...,...,...
19967,20773.0,NaN,NaN,NaN,NaN,1 (11) 500 555-0171
19968,20774.0,NaN,NaN,NaN,NaN,910-555-0166
19969,20775.0,NaN,NaN,NaN,NaN,813-555-0148
19970,20776.0,NaN,NaN,NaN,NaN,1 (11) 500 555-0171
